# Define data/result directory

In [6]:
import os

data_folder = "../data"
result_folder = "../results"
manga_list = "vi/Ruri Dragon (Oneshot)/Ch. None"  # Will be changed to list later

# Ruri Dragon
manga_folder = os.path.join(data_folder, manga_list)

individual_result_folder = os.path.join(result_folder, manga_list)
json_output_dir = os.path.join(individual_result_folder, "json_results")
result_image_output_dir = os.path.join(individual_result_folder, "image_results")

cut_texts_dir = os.path.join(individual_result_folder, "cut_texts")

raw_images = os.listdir(manga_folder)
json_files = os.listdir(json_output_dir)
easy_ocr_viet_ocr_result_dir = os.path.join(individual_result_folder, "easy_ocr_viet_ocr_result")
os.makedirs(easy_ocr_viet_ocr_result_dir, exist_ok=True)

# Initialize models

In [3]:
import easyocr
from vietocr.tool.config import Cfg

# Initialize EasyOCR (Vietnamese)
reader = easyocr.Reader(['vi'])

config = Cfg.load_config_from_name("vgg_transformer") # Load YAML config file
from vietocr.tool.predictor import Predictor
config['weights'] = "../models/viet_ocr/pretrained_weight/vgg_transformer.pth"
# config['weights'] = "../models/viet_ocr/custom_weight/transformerocr-1.pth"
config['cnn']['pretrained']=False
detector = Predictor(config)

d:\miniconda3\envs\vietocr\lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
import os
import json
from PIL import Image

for json_file in json_files:
    if not json_file.endswith(".json"):
        continue

    base_name = os.path.splitext(json_file)[0]
    json_path = os.path.join(json_output_dir, json_file)
    cut_page_dir = os.path.join(cut_texts_dir, base_name)

    if not os.path.exists(cut_page_dir):
        print(f"⚠️ No cut_texts found for {json_file}, skipping...")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    new_ocr = []
    for idx, (bbox, is_essential) in enumerate(zip(data["texts"], data["is_essential_text"])):
        if not is_essential:
            new_ocr.append(data["ocr"][idx])
            continue

        cut_img_path = os.path.join(cut_page_dir, f"{base_name}_{idx:03}.png")
        if not os.path.exists(cut_img_path):
            print(f"⚠️ Missing cut image {cut_img_path}, keeping original OCR.")
            new_ocr.append(data["ocr"][idx])
            continue

        try:
            # Step 1: Detect text regions with EasyOCR (detail=1 gives bbox + text + confidence)
            results = reader.readtext(cut_img_path, detail=1, paragraph=False)

            if not results:
                raise ValueError("No text detected")

            line_texts = []
            for (coords, _, _) in results:  # coords = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
                x_min = min([pt[0] for pt in coords])
                y_min = min([pt[1] for pt in coords])
                x_max = max([pt[0] for pt in coords])
                y_max = max([pt[1] for pt in coords])

                # crop the detected line
                img = Image.open(cut_img_path).convert("RGB")
                cropped = img.crop((x_min, y_min, x_max, y_max))

                # Step 2: Recognize with VietOCR
                try:
                    text_viet = detector.predict(cropped)
                except Exception as e:
                    print(f"⚠️ VietOCR failed on line in {cut_img_path}: {e}")
                    text_viet = ""

                if text_viet.strip():
                    line_texts.append(text_viet)

            # Merge detected lines into one text block
            vi_text = " ".join(line_texts) if line_texts else data["ocr"][idx]

        except Exception as e:
            print(f"❌ OCR pipeline failed for {cut_img_path}: {e}")
            vi_text = data["ocr"][idx]

        new_ocr.append(vi_text)

    data["ocr"] = new_ocr

    out_path = os.path.join(easy_ocr_viet_ocr_result_dir, json_file)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for {json_file} → {out_path}")


⚠️ No cut_texts found for 00.json, skipping...
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 01.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\01.json
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 02.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\02.json
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 03.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\03.json
❌ OCR pipeline failed for ../results\vi/Ruri Dragon (Oneshot)/Ch. None\cut_texts\04\text_004.png: No text detected
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 04.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\04.json
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 05.json → ../results\vi/Ruri Dragon (Oneshot)/Ch. None\easy_ocr_viet_ocr_result\05.json
✅ Saved Combined (EasyOCR-coords + VietOCR-text) result for 06.json → ../results\vi/Ruri Dragon (O